In [28]:
using DynamicalSystems
using DifferentialEquations

In [2]:
# Parameters
num_steps = 1000  # number of time steps in the simulation
dt = 0.1  # step size
pursuer_max_speed = 5.0  # maximum speed of the pursuer
pursuer_max_ang_velocity = 1.0  # maximum angular velocity of the pursuer
evader_max_accel = 2.0  # maximum acceleration of the evader
evader_max_speed = 10.0  # maximum speed of the evader
pursuer_obs_noise = 0.1  # variance of the observation noise for the pursuer
evader_obs_noise = 0.1  # variance of the observation noise for the evader

0.1

In [3]:
num_steps 

1000

In [4]:
dt

0.1

In [5]:
typeof(dt)

Float64

In [6]:
vector = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [7]:
vector[1]

1

In [8]:
vector[1]= 10

10

In [9]:
vector[1]

10

In [10]:
vector

5-element Vector{Int64}:
 10
  2
  3
  4
  5

In [11]:
function evader_dynamics!(dx, x, p, t)
    # Unpack state variables
    ex, ey, evx, evy = x
    # Unpack parameters
    max_accel, max_speed, pursuer_pos, pursuer_vel, obs_noise = p
    
    # Calculate distance and bearing to pursuer
    dx = pursuer_pos[1] - ex
    dy = pursuer_pos[2] - ey
    r = hypot(dx, dy)
    theta = atan(dy, dx)
    
    # Apply observation noise
    r += randn() * obs_noise
    theta += randn() * obs_noise
    
    # Estimate pursuer position and velocity
    pursuer_vx_est = pursuer_vel[1] + randn() * obs_noise
    pursuer_vy_est = pursuer_vel[2] + randn() * obs_noise
    pursuer_px_est = pursuer_pos[1] + pursuer_vx_est * r / evader_max_speed
    pursuer_py_est = pursuer_pos[2] + pursuer_vy_est * r / evader_max_speed
    
    # Control law to maximize distance to pursuer
    desired_accel = max_accel .* (cos(theta), sin(theta)) #reworked to handle scalar times vector
    dax = desired_accel[1]
    day = desired_accel[2]
    accel_error = (dax-evx, day-evy)
    accel = norm(accel_error) <= max_accel ? accel_error : normalize(accel_error) * max_accel
    speed_error = max_speed - hypot(evx, evy)
    speed = clamp(speed_error, -max_accel * dt, max_accel * dt) + hypot(evx, evy)
    
    # Update state variables
    dx[1] = evx
    dx[2] = evy
    dx[3] = accel[1]
    dx[4] = accel[2]
end


evader_dynamics! (generic function with 1 method)

In [ ]:

# Helper function to wrap an angle to the range [-pi, pi]
# this was missing from the GPT3 recommendation so i added it
function wrap_to_pi(angle)
    return atan(sin(angle), cos(angle))
end

In [29]:
show(evader_dynamics!)

evader_dynamics!

In [54]:
# Set up input arguments
dx = zeros(4)  # initialize the state variable derivatives
x = [0.0, 0.0, 0.0, 0.0]  # initialize the state variables
p = [2.0, 5.0, [10.0, 10.0], [0.0, 0.0], 0.1]  # initialize the parameters
t = 0.0  # initialize the time variable

0.0

In [55]:
ex, ey, evx, evy = x

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [56]:
ex

0.0

In [57]:
max_accel, max_speed, pursuer_pos, pursuer_vel, obs_noise = p

5-element Vector{Any}:
 2.0
 5.0
  [10.0, 10.0]
  [0.0, 0.0]
 0.1

In [58]:
dx = pursuer_pos[1] - ex

10.0

In [35]:
dy = pursuer_pos[2] - ey

10.0

In [36]:
r = hypot(dx, dy)

14.142135623730951

In [37]:
theta = atan(dy, dx)

0.7853981633974483

In [38]:
    # Apply observation noise
    r += randn() * obs_noise
    theta += randn() * obs_noise
    
    # Estimate pursuer position and velocity
    pursuer_vx_est = pursuer_vel[1] + randn() * obs_noise
    pursuer_vy_est = pursuer_vel[2] + randn() * obs_noise
    pursuer_px_est = pursuer_pos[1] + pursuer_vx_est * r / evader_max_speed
    pursuer_py_est = pursuer_pos[2] + pursuer_vy_est * r / evader_max_speed

10.038525127674857

In [39]:
function clamp_vector2magnitude(v::AbstractVector{T}, max_mag::Real) where {T <: Real}
    v_mag = norm(v)
    if v_mag <= max_mag
        return copy(v)
    else
        return (max_mag / v_mag) .* v
    end
end

clamp_vector2magnitude (generic function with 1 method)

In [40]:
    # Control law to maximize distance to pursuer
    desired_accel = max_accel .* (cos(theta), sin(theta))


(1.405163647939234, 1.4232059311674137)

In [41]:
using LinearAlgebra


In [42]:
desired_accel = max_accel .* (cos(theta), sin(theta))
dax = desired_accel[1]
day = desired_accel[2]
accel_error = (dax-evx, day-evy)
accel = norm(accel_error) <= max_accel ? accel_error : normalize(accel_error) * max_accel
speed_error = max_speed - hypot(evx, evy)
speed = clamp(speed_error, -max_accel * dt, max_accel * dt) + hypot(evx, evy)

0.2

In [53]:
dx

10.0

In [49]:
    # Update state variables
    dx[1] = evx


MethodError: MethodError: no method matching setindex!(::Float64, ::Float64, ::Int64)

In [ ]:
    dx[2] = evy
    dx[3] = accel[1]
    dx[4] = accel[2]

In [47]:
# Call the function
evader_dynamics!(dx, x, p, t)


MethodError: MethodError: no method matching -(::Tuple{Float64, Float64}, ::Tuple{Float64, Float64})
Closest candidates are:
  -(!Matched::VectorizationBase.CartesianVIndex, ::Any) at ~/.julia/packages/VectorizationBase/zlsw9/src/cartesianvindex.jl:80
  -(!Matched::ChainRulesCore.AbstractThunk, ::Any) at ~/.julia/packages/ChainRulesCore/a4mIA/src/tangent_types/thunks.jl:34
  -(!Matched::IntervalRootFinding.Slope, ::Any) at ~/.julia/packages/IntervalRootFinding/gKaGR/src/slopes.jl:75
  ...